In [1]:
import pandas as pd
from numpy import log

In [2]:
df = pd.read_csv("../data/raw/probability_of_default.csv")

In [3]:
df.describe().loc[['min', 'mean', 'max']].transpose()

,min,mean,max
person_age,20.00,27.734600,144.00
person_income,4000.00,66074.848470,6000000.00
person_emp_length,0.00,4.789686,123.00
loan_amnt,500.00,9589.371106,35000.00
loan_int_rate,5.42,11.011695,23.22
loan_status,0.00,0.218164,1.00
loan_percent_income,0.00,0.170203,0.83
cb_person_cred_hist_length,2.00,5.804211,30.00


In [4]:
df.drop(columns=['loan_intent', 'loan_grade'], inplace=True)

In [5]:
df.rename(columns={
    'person_age': 'age',
    'person_income': 'income',
    'person_home_ownership': 'home_ownership_type',
    'person_emp_length': 'employment_length',
    'loan_amnt': 'loan_amount',
    'loan_int_rate': 'loan_interest_rate',
    'loan_status': 'is_default',
    'cb_person_default_on_file': 'has_defaulted_before',
    'cb_person_cred_hist_length': 'credit_history_length',
    'loan_percent_income': 'loan_to_income_ratio',
}, inplace=True)

In [6]:
df.head()

,age,income,home_ownership_type,employment_length,loan_amount,loan_interest_rate,is_default,loan_to_income_ratio,has_defaulted_before,credit_history_length
0,22,59000,RENT,123.0,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,35000,14.27,1,0.55,Y,4


In [7]:
df = df[(df['age'] < 100) & (df['employment_length'] < 80)]  # Filter out unrealistic ages and employment_length

df['has_defaulted_before'] = df['has_defaulted_before'].map({'Y': 1, 'N': 0})

home_ownership_score = {'OWN': 1.2,'MORTGAGE': 1.1,'RENT': 1.0,'OTHER': 0.9}
home_ownership_score = df['home_ownership_type'].map(home_ownership_score)

employment_lenght_score = log(1 + df['employment_length'])

df['debt_to_stability_ratio'] = (df['loan_amount'] / (df['income'] * home_ownership_score * employment_lenght_score)).round(4)

df.head()

,age,income,home_ownership_type,employment_length,loan_amount,loan_interest_rate,is_default,loan_to_income_ratio,has_defaulted_before,credit_history_length,debt_to_stability_ratio
1,21,9600,OWN,5.0,1000,11.14,0,0.10,0,2,0.0484
2,25,9600,MORTGAGE,1.0,5500,12.87,1,0.57,0,3,0.7514
3,23,65500,RENT,4.0,35000,15.23,1,0.53,0,2,0.3320
4,24,54400,RENT,8.0,35000,14.27,1,0.55,1,4,0.2928
5,21,9900,OWN,2.0,2500,7.14,1,0.25,0,2,0.1915


In [8]:
df.to_csv("../data/processed/probability_of_default.csv", index=False)